# 🎯 BillingRag - Databricks Setup Notebook

**Run this notebook in Databricks to set up everything you need.**

## What this does:
1. Creates the schema
2. Creates the 2 tables (rag_documents, hvs_data)
3. Inserts sample data for testing
4. Verifies everything works

---

## 📋 Step 1: Configuration

Change these values to match your setup:

In [ ]:
# Configuration - CHANGE THESE!
CATALOG = "main"  # or your catalog name
SCHEMA = "billingrag_tatsat"  # your schema name

# Full table names
RAG_TABLE = f"{CATALOG}.{SCHEMA}.rag_documents"
HVS_TABLE = f"{CATALOG}.{SCHEMA}.teleco_data"

print(f"📍 Catalog: {CATALOG}")
print(f"📍 Schema: {SCHEMA}")
print(f"📍 RAG Table: {RAG_TABLE}")
print(f"📍 HVS Table: {HVS_TABLE}")

## 📋 Step 2: Create Schema

In [ ]:
# Create schema if it doesn't exist
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")
print(f"✅ Schema {CATALOG}.{SCHEMA} created/exists")

## 📋 Step 3: Create RAG Documents Table

This table stores your knowledge base:
- `text` - Document chunks
- `embedding` - Vector embeddings (384 dimensions)

In [ ]:
# Create RAG Documents table
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {RAG_TABLE} (
    id STRING NOT NULL COMMENT 'Unique chunk ID (UUID)',
    filename STRING COMMENT 'Original source filename',
    text STRING COMMENT 'Chunk text content',
    embedding ARRAY<FLOAT> COMMENT 'Vector embedding (384 dimensions)',
    chunk_index INT COMMENT 'Position of chunk in original document',
    upload_date TIMESTAMP COMMENT 'When this chunk was uploaded',
    metadata MAP<STRING, STRING> COMMENT 'Additional metadata like source, tags'
) 
USING DELTA
COMMENT 'RAG knowledge base - document chunks with vector embeddings for semantic search'
""")

print(f"✅ Table {RAG_TABLE} created/exists")

## 📋 Step 4: Create HVS Data Table

This table stores billing data for ML predictions:
- Enterprise/subaccount billing rates
- Used to train the prediction model

In [ ]:
# Create HVS Data table
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {HVS_TABLE} (
    id STRING NOT NULL COMMENT 'Unique record ID (UUID)',
    enterprise_id STRING COMMENT 'Enterprise/company identifier',
    subaccount STRING COMMENT 'Subaccount identifier within enterprise',
    device_type STRING COMMENT 'Device type: ROUTER, SWITCH, FIREWALL, etc',
    service_type STRING COMMENT 'Service type: INTERNET, VPN, MPLS, SD-WAN',
    monthly_rate DECIMAL(10,2) COMMENT 'Monthly billing rate in dollars',
    bandwidth_mbps INT COMMENT 'Provisioned bandwidth in Mbps',
    contract_start DATE COMMENT 'Contract start date',
    contract_end DATE COMMENT 'Contract end date',
    status STRING COMMENT 'Status: ACTIVE, INACTIVE, SUSPENDED',
    upload_date TIMESTAMP COMMENT 'When this record was uploaded to system',
    data_month STRING COMMENT 'Source data month in YYYY-MM format'
)
USING DELTA
COMMENT 'HVS billing data for ML rate predictions - retains 3-4 months of history'
""")

print(f"✅ Table {HVS_TABLE} created/exists")

## 📋 Step 5: Verify Tables

In [ ]:
# Show all tables in schema
tables = spark.sql(f"SHOW TABLES IN {CATALOG}.{SCHEMA}")
display(tables)

print("\n📊 Table Details:")
for table in [RAG_TABLE, HVS_TABLE]:
    count = spark.sql(f"SELECT COUNT(*) as cnt FROM {table}").first()['cnt']
    print(f"   {table}: {count} rows")

## 📋 Step 6: Insert Sample Data (Optional)

Run this to add test data for development:

In [ ]:
# Insert sample RAG document
import uuid
from datetime import datetime

sample_doc_id = str(uuid.uuid4())
sample_embedding = [0.1] * 384  # Mock 384-dimension embedding

spark.sql(f"""
INSERT INTO {RAG_TABLE}
VALUES (
    '{sample_doc_id}',
    'sample_billing_policy.txt',
    'This is a sample billing policy document. Enterprise billing rates are calculated based on bandwidth consumption, service type, and contract terms. Standard rates apply for INTERNET services while premium rates apply for MPLS and SD-WAN services.',
    array({','.join(map(str, sample_embedding))}),
    0,
    current_timestamp(),
    map('source', 'sample', 'type', 'policy')
)
""")

print(f"✅ Sample document inserted: {sample_doc_id}")

In [ ]:
# Insert sample HVS data
import uuid

sample_hvs_data = [
    ('ENT001', 'SUB001', 'ROUTER', 'INTERNET', 250.00, 100, '2025-01-01', '2026-01-01', 'ACTIVE'),
    ('ENT001', 'SUB002', 'SWITCH', 'VPN', 175.50, 50, '2025-02-01', '2026-02-01', 'ACTIVE'),
    ('ENT001', 'SUB003', 'FIREWALL', 'INTERNET', 320.00, 200, '2025-01-15', '2026-01-15', 'ACTIVE'),
    ('ENT002', 'SUB001', 'ROUTER', 'MPLS', 500.00, 500, '2024-06-01', '2025-06-01', 'ACTIVE'),
    ('ENT002', 'SUB002', 'ROUTER', 'SD-WAN', 450.00, 300, '2024-09-01', '2025-09-01', 'ACTIVE'),
    ('ENT003', 'SUB001', 'SWITCH', 'INTERNET', 125.00, 50, '2025-03-01', '2026-03-01', 'ACTIVE'),
    ('ENT003', 'SUB002', 'ROUTER', 'VPN', 200.00, 100, '2025-01-01', '2026-01-01', 'ACTIVE'),
    ('ENT004', 'SUB001', 'FIREWALL', 'MPLS', 600.00, 1000, '2024-12-01', '2025-12-01', 'ACTIVE'),
    ('ENT004', 'SUB002', 'ROUTER', 'INTERNET', 180.00, 100, '2025-02-01', '2026-02-01', 'ACTIVE'),
    ('ENT005', 'SUB001', 'ROUTER', 'SD-WAN', 350.00, 200, '2025-01-01', '2026-01-01', 'ACTIVE'),
]

for ent, sub, dev, svc, rate, bw, start, end, status in sample_hvs_data:
    record_id = str(uuid.uuid4())
    spark.sql(f"""
    INSERT INTO {HVS_TABLE}
    VALUES (
        '{record_id}',
        '{ent}',
        '{sub}',
        '{dev}',
        '{svc}',
        {rate},
        {bw},
        '{start}',
        '{end}',
        '{status}',
        current_timestamp(),
        '2025-01'
    )
    """)

print(f"✅ Inserted {len(sample_hvs_data)} sample HVS records")

## 📋 Step 7: Query Test Data

In [ ]:
# Check RAG documents
print("📄 RAG Documents:")
display(spark.sql(f"SELECT id, filename, LEFT(text, 100) as text_preview, upload_date FROM {RAG_TABLE} LIMIT 5"))

In [ ]:
# Check HVS data
print("💰 HVS Data:")
display(spark.sql(f"""
    SELECT enterprise_id, subaccount, device_type, service_type, monthly_rate, bandwidth_mbps, status
    FROM {HVS_TABLE}
    ORDER BY enterprise_id, subaccount
    LIMIT 20
"""))

In [ ]:
# HVS Summary Statistics
print("📊 HVS Summary:")
display(spark.sql(f"""
    SELECT 
        COUNT(*) as total_records,
        COUNT(DISTINCT enterprise_id) as unique_enterprises,
        COUNT(DISTINCT subaccount) as unique_subaccounts,
        AVG(monthly_rate) as avg_rate,
        MIN(monthly_rate) as min_rate,
        MAX(monthly_rate) as max_rate,
        AVG(bandwidth_mbps) as avg_bandwidth
    FROM {HVS_TABLE}
"""))

## ✅ Setup Complete!

You now have:
1. **Schema**: `main.billingrag_tatsat`
2. **RAG Table**: `rag_documents` - for knowledge base
3. **HVS Table**: `hvs_data` - for ML predictions
4. **Sample Data**: Ready for testing

### Next Steps:
1. Copy your Databricks credentials
2. Update `.env` file in your local project
3. Run `python run.py test` to verify connection
4. Run `python run.py serve` to start the app

### Connection Details You Need:
- **Host**: Your workspace URL (e.g., `https://xxx.cloud.databricks.com`)
- **Token**: Generate from User Settings → Developer → Access Tokens
- **HTTP Path**: From SQL Warehouse → Connection Details

In [ ]:
# Print connection info (DON'T SHARE TOKEN!)
import os

print("="*50)
print("📋 YOUR CONFIGURATION (for .env file)")
print("="*50)
print(f"""
# Copy these to your .env file:

DATABRICKS_HOST={spark.conf.get('spark.databricks.workspaceUrl', 'YOUR_WORKSPACE_URL')}
DATABRICKS_TOKEN=dapi_your_token_here
DATABRICKS_HTTP_PATH=/sql/1.0/warehouses/YOUR_WAREHOUSE_ID

DATABRICKS_CATALOG={CATALOG}
DATABRICKS_SCHEMA={SCHEMA}
""")